TODO:
- Reproduce the error for image loading
- Assert cam_R_w2c with reprojection

In [4]:
import os
import time
import json

from collections import OrderedDict
import yaml
import argparse

import torch
import numpy as np
import pandas as pd
import pickle as pkl
import logging
from cosypose.config import EXP_DIR, MEMORY, RESULTS_DIR, LOCAL_DATA_DIR

import cosypose.utils.tensor_collection as tc

In [ ]:
n_workers = 8
n_plotters = 8
n_views = 1

n_frames = None
scene_id = None
group_id = None
n_groups = None
n_views = args.n_views
skip_mv = args.n_views < 2
skip_predictions = False

object_set = 'bracket_assembly'
coarse_run_id = f'bracket_assembly_coarse-transnoise-zxyavg-616093'
refiner_run_id = 'bracket_assembly_refiner--558735'
n_coarse_iterations = 1
n_refiner_iterations = 2

In [5]:
def load_custom_detection_from_gt():
    path_data_dir = LOCAL_DATA_DIR / 'bop_datasets' / 'bracket_assembly'
    path_scene_dir = os.path.join(path_data_dir, "train_pbr")
    scene_names = os.listdir(path_scene_dir)
    infos, poses, bboxes = [], [], []
    # debug_only = 0
    for scene_id, scene_name in enumerate(scene_names):
        path_scene_gt_info = os.path.join(path_scene_dir, scene_name, "scene_gt_info.json")
        path_scene_gt = os.path.join(path_scene_dir, scene_name, "scene_gt.json")
        path_scene_gt_camera = os.path.join(path_scene_dir, scene_name, "scene_camera.json")
        with open(path_scene_gt_info, "r") as f:
            json_data_gt_info = json.load(f)
        with open(path_scene_gt, "r") as f:
            json_data_gt = json.load(f)
        with open(path_scene_gt_camera, "r") as f:
            json_gt_camera = json.load(f)
        img_names_rgb = os.listdir(os.path.join(path_scene_dir, scene_name, "rgb"))
        # todo
        # scene_camera = load_scene_camera(path_scene_gt_camera)
        from cosypose.lib3d import Transform
        for img_id, img_name in enumerate(img_names_rgb[:-1]):
            if not f"{img_id}" in json_data_gt_info:
                continue
            if not f"{img_id}" in json_data_gt:
                continue
            if not f"{img_id}" in json_gt_camera:
                continue
            # cam_R_w2c = scene_camera[img_id]["cam_R_w2c"]
            # TODO
            cam_R_w2c = json_gt_camera[f"{img_id}"]["cam_R_w2c"]
            cam_t_w2c = json_gt_camera[f"{img_id}"]["cam_t_w2c"]
            row0 = [cam_R_w2c[0], cam_R_w2c[1], cam_R_w2c[2], cam_t_w2c[0]] 
            row1 = [cam_R_w2c[3], cam_R_w2c[4], cam_R_w2c[5], cam_t_w2c[1]]
            row2 = [cam_R_w2c[6], cam_R_w2c[7], cam_R_w2c[8], cam_t_w2c[2]]           
            row3 = [0, 0, 0, 1]
            cam_rot_loc_mat = np.asarray([row0, row1, row2, row3])


            if 'cam_R_w2c' in json_gt_camera[f"{img_id}"]:
                RC0 = np.array(json_gt_camera[f"{img_id}"]['cam_R_w2c']).reshape(3, 3)
                tC0 = np.array(json_gt_camera[f"{img_id}"]['cam_t_w2c'])# * 0.001
                TC0 = Transform(RC0, tC0)
                T0C = TC0.inverse()
                T0C = T0C.toHomogeneousMatrix()
            for label_idx, label in enumerate(json_data_gt[f"{img_id}"]):
                

                obj_id = label["obj_id"] # int
                list_bbox = json_data_gt_info[f"{img_id}"][label_idx]["bbox_obj"] # TODO: ?

            # list_bbox = json_data_gt_info[f"{img_id}"][0]["bbox_obj"] # TODO: ?
                xmin = list_bbox[0]
                ymin = list_bbox[1]
                xmax = list_bbox[0] + list_bbox[2]
                ymax = list_bbox[1] + list_bbox[3]
                list_bbox = [xmin, ymin, xmax, ymax]
                list_rot  = json_data_gt[f"{img_id}"][label_idx]["cam_R_m2c"]
                list_loc  = json_data_gt[f"{img_id}"][label_idx]["cam_t_m2c"]
                # RCO = np.array(json_data_gt[f"{img_id}"][label_idx]['cam_R_m2c']).reshape(3, 3)
                # tCO = np.array(json_data_gt[f"{img_id}"][label_idx]['cam_t_m2c']) #* 0.001
                # TCO = Transform(RCO, tCO)
                # T0O = T0C * TCO
                # T0O = T0O.toHomogeneousMatrix()

                row0 = [list_rot[0], list_rot[1], list_rot[2], list_loc[0]] 
                row1 = [list_rot[3], list_rot[4], list_rot[5], list_loc[1]]
                row2 = [list_rot[6], list_rot[7], list_rot[8], list_loc[2]]
                row3 = [0, 0, 0, 1]
                rot_loc_mat = np.asarray([row0, row1, row2, row3])
                rot_loc_mat = np.matmul(np.linalg.inv(cam_rot_loc_mat), rot_loc_mat)
                if scene_id == 0 and img_id == 20:
                    print("label_idx",label_idx)
                    print("rot_loc_mat", rot_loc_mat)
                infos.append(dict(
                        scene_id=scene_id,
                        view_id=img_id,
                        score=1,
                        label=f"obj_{obj_id:06d}",
                    ))
                # poses.append(T0O)
                poses.append(rot_loc_mat)
                bboxes.append(list_bbox)
    data = tc.PandasTensorCollection(
        infos=pd.DataFrame(infos),
        poses=torch.as_tensor(np.stack(poses)).float(),
        bboxes=torch.as_tensor(np.stack(bboxes)).float(),
    ).cpu()
    return data
bracket_detections = load_custom_detection_from_gt().cpu()

label_idx 0
rot_loc_mat [[-0.99587986  0.07700412 -0.04789164 -0.11810839]
 [ 0.04993979  0.02489411 -0.99844195  0.07891538]
 [-0.07569207 -0.99672006 -0.02863722  0.03211115]
 [ 0.          0.          0.          1.        ]]
label_idx 1
rot_loc_mat [[-0.76424811 -0.61595642  0.19110889  0.00223391]
 [-0.64437037  0.74155461 -0.18677107  0.01961104]
 [-0.02667499 -0.26588431 -0.96363596  0.03832466]
 [ 0.          0.          0.          1.        ]]
label_idx 2
rot_loc_mat [[-0.6886958   0.71697526  0.1079091   0.01520633]
 [-0.67750345 -0.689374    0.25642217 -0.01933644]
 [ 0.25823804  0.1034881   0.9605224   0.00227919]
 [ 0.          0.          0.          1.        ]]
label_idx 3
rot_loc_mat [[ 0.73763925 -0.6749509  -0.01815473  0.02072153]
 [-0.67493606 -0.73634533 -0.04750498  0.01430454]
 [ 0.01869533  0.04729474 -0.99870608  0.02342988]
 [ 0.          0.          0.          1.        ]]
label_idx 4
rot_loc_mat [[ 0.99575101 -0.05363404 -0.07485447  0.00574114]
 [ 0.0411

In [6]:
len(bracket_detections)

14917

In [ ]:
scene_id = 0
frame_id = 20
rgb = open('...')
meshes = self.mesh_db.select(labels)
points = meshes.sample_points(2000, deterministic=True)
uv = project_points(points, K, TCO)
boxes_rend = boxes_from_uv(uv)
boxes_crop, images_cropped = deepim_crops(
    images=images, obs_boxes=boxes_rend, K=K,
    TCO_pred=TCO, O_vertices=points, output_size=self.render_size, lamb=1.4
)
plot(image)
plot(boxes)